In [1]:
%load_ext tensorboard
import datetime

import tensorflow as tf
import numpy as np

import dataset
import gpu_utils
gpu_utils.setup_gpus()

BUFFER_SIZE = 10000
BATCH_SIZE = 512  # 512

data = dataset.load_bible(BATCH_SIZE, validation=0.2)

1 Physical GPUs, 1 Logical GPUs


In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(256, return_sequences=True, recurrent_initializer='glorot_uniform'),
        input_shape=(data.maxlen, data.letters_size),
        batch_size=BATCH_SIZE
    ),
    tf.keras.layers.GRU(data.niqqud_size, return_sequences=True, recurrent_initializer='glorot_uniform'),
])

# best: mean_squared_logarithmic_error, stateless

model.compile(loss='mean_squared_logarithmic_error',
              optimizer='adam',
              metrics=['accuracy'])
model.build()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (512, 237, 512)           456192    
_________________________________________________________________
gru_1 (GRU)                  (512, 237, 15)            23805     
Total params: 479,997
Trainable params: 479,997
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.load_weights(tf.train.latest_checkpoint('niqqud_checkpoints/'))

In [8]:
%tensorboard --logdir logs\fit

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 2620.

In [9]:
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(data.input_texts, data.niqqud_texts,
          batch_size=BATCH_SIZE,
          epochs=10,
          validation_data=(data.input_validation, data.niqqud_validation),
          callbacks=[
              # tf.keras.callbacks.ModelCheckpoint(filepath='niqqud_checkpoints/ckpt_{epoch}', save_weights_only=True),
              tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1),
              tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=0, min_lr=0.001),
              tensorboard_callback,
          ]
          )

Train on 16384 samples, validate on 4096 samples
Epoch 1/10
16384/16384 [==============================] - 7s 451us/sample - loss: 0.0286 - accuracy: 0.9285 - val_loss: 0.0286 - val_accuracy: 0.9299
Epoch 2/10
16384/16384 [==============================] - 7s 440us/sample - loss: 0.0286 - accuracy: 0.9285 - val_loss: 0.0286 - val_accuracy: 0.9298
Epoch 3/10
16384/16384 [==============================] - 7s 443us/sample - loss: 0.0286 - accuracy: 0.9285 - val_loss: 0.0286 - val_accuracy: 0.9299
Epoch 4/10
16384/16384 [==============================] - 7s 441us/sample - loss: 0.0286 - accuracy: 0.9286 - val_loss: 0.0286 - val_accuracy: 0.9300
Epoch 5/10
16384/16384 [==============================] - 7s 447us/sample - loss: 0.0286 - accuracy: 0.9286 - val_loss: 0.0286 - val_accuracy: 0.9299
Epoch 6/10
16384/16384 [==============================] - 7s 444us/sample - loss: 0.0286 - accuracy: 0.9286 - val_loss: 0.0286 - val_accuracy: 0.9300
Epoch 7/10
16384/16384 [===========================

In [10]:
model.add(tf.keras.layers.Softmax())

def print_predictions(batch):
    results = data.merge(batch, model.predict(batch))

    for r in results:
        print(r)


print_predictions(data.input_texts[0:BATCH_SIZE])

וְאֶת-גְבִיעִי גְבִיע הַכָסֶף תָשִים בְפִי אֲמִתַחַת הַקָטָן וְאֶת כָסֶף שְבָרוּ וַיַעַש כְדְבֶר יוֹסֶף אֲשֶר דָבָר
וַיְאמֶר לִי הַמֶלֶךְ עַל-מָה-זָה אָתָה מִבְקֶש וְאִתְפְלֵל אֶל-אֵלְהֵי הַשָמִים
בֶת-בְבֵל הַשְדוֹדָה אֲשְרִי שִישְלֶם-לֶךְ אֶת-גְמוֹלֶךְ שְגְמִלֶת לָנוֹ
וַיִשְמַע זְרְבְבָל בֶן-שְלְתִיאל וַיְהוּשָע בֶן-יְהוּצְדָק הַכָהֵן הַגְדוֹל וְכָל שְאֲרִית הַעָם בְקוֹל יְהוָה אַלְהֵיהֶם וְעַל-דִבְרִי חֲגִי הַנָבִיא כָאֲשֶר שְלְחוֹ יְהוָה אַלְהֵיהֶם וַיִירָאוּ הַעָם מִפְנֵי יְהוָה
וְרוַח יְהוָה לְבְשָה אֶת-גְדְעוֹן וַיִתְקַע בְשוֹפָר וַיִזְעַק אֲבִיעֲזֶר אַחָרָיוּ
וּמִצְרִים אָדֶם וְלָא-אָל וְסוֹסֵיהֶם בְשֶר וְלָא-רוֹח וַיְהוָה יְטָה יָדוּ וְכָשֶל עוֹזֶר וְנָפָל עֲזֶר וַיַחָדוּ כְלֶם יִכְלְיוֹן
כָה אֲמֶר יְהוָה צְבְאוֹת קְנָאתִי לְצִיוֹן קְנָאָה גְדוֹלָה וַחָמָה גְדוֹלָה קְנָאתִי לָה
כִי אֶם-הַיטִיב תִיטִיבוּ אֶת-דַרְכֵיכָם וְאֶת-מַעַלְלֵיכָם אֶם-עָשוּ תַעֲשוּ מִשְפָט בִין אִיש וְבִין רָעָהוּ
וּבְנוּ יִתְגָרוּ וְאִסְפוּ הַמוֹן חַילִים רָבִים וְבָא בוֹא וְשָטֶף וַעָבֶר וַיִשֶב וַיִתְג

In [ ]:
del model